In [11]:
import os
import pandas
import argparse
import time
import re
import pickle
import def_function
from def_class import *

In [13]:
__version__ = "V3.4(Editor) 2023-08-07"

In [15]:
# 前置参数-debug
counts_cutoff = 2

annotation_col = ["gene_id", "transcript_id", "gene_name"]
quantification_col = ["annot_gene_id", "annot_transcript_id","annot_transcript_name"]

absolute_path = False
file_path = "F:/OneDrive/Master/Project/trans/data/"
input_sample_info = "0000_sample_info.tsv"
input_gene_annotation_file = "Homo_sapiens.GRCh38.109.gtf.gz"
output_df_filename = "0001_total_info.tsv"
output_pickle_filename = "0001_total_info.pickle"

In [ ]:
# 前置参数
parser = argparse.ArgumentParser()
parser.add_argument("--counts_cutoff", dest="counts_cutoff", required=False, type=int, default=2, help="=2,\t the cutoff_value of transcript in quantification file")
parser.add_argument("--annotation_col", dest="annotation_col", required=False, type=str, default=["gene_id", "transcript_id", "gene_name"], nargs="*", help="=[\"gene_id\", \"transcript_id\", \"gene_name\"],\t selected columns of annotation file")
parser.add_argument("--quantification_col", dest="quantification_col", required=False, type=str, default=["annot_gene_id", "annot_transcript_id","annot_transcript_name"], nargs="*", help="=[\"annot_gene_id\", \"annot_transcript_id\",\"annot_transcript_name\"],\t selected columns of quantification file")
parser.add_argument("--absolute_path", dest="absolute_path", required=False, action="store_true", default=False, help="use absolute path")
parser.add_argument("--file_path", dest="file_path", required=False, type=str, default="./", help="the dir of data, if absolute_path is False")
parser.add_argument("--input_sample_info", dest="input_sample_info", required=False, type=str, default="0000_sample_info.tsv", help="=0000_sample_info.tsv,\t the info file of input sample")
parser.add_argument("--input_gene_annotation_file", dest="input_gene_annotation_file", required=False, type=str, default="Homo_sapiens.GRCh38.109.gtf.gz", help="\"Homo_sapiens.GRCh38.109.gtf.gz\",\t genome annotation file path")
parser.add_argument("--output_df_filename", dest="output_df_filename", required=False, type=str, default="0001_total_info.tsv", help="=,\t the output df file of samples")
parser.add_argument("--output_pickle_filename", dest="output_pickle_filename", required=False, type=str, default="0001_total_info.pickle", help="=,\t the output pickle file of samples")

args = parser.parse_args()
counts_cutoff = args.counts_cutoff
annotation_col = args.annotation_col
quantification_col = args.quantification_col
absolute_path = args.absolute_path
file_path = args.file_path
input_sample_info = args.input_sample_info
input_gene_annotation_file = args.input_gene_annotation_file
output_df_filename = args.output_df_filename
output_pickle_filename = args.output_pickle_filename

In [16]:
# 补全路径
if absolute_path == False:
    input_sample_info = "{}{}".format(file_path, input_sample_info)
    input_gene_annotation_file = "{}raw_data/annotation/{}".format(file_path, input_gene_annotation_file)
    output_df_filename = "{}{}".format(file_path, output_df_filename)
    output_pickle_filename = "{}{}".format(file_path, output_pickle_filename)

In [17]:
# 后置参数
# 获取样本名
sample_info = pandas.read_csv(input_sample_info, sep='\t')
sample_name_list = sample_info["GEO_accession"].to_list()

In [ ]:
# 打印参数
print('\n')
print("[Date]{}".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))))
print("[Script]script: {}".format(__file__))
print("[Version]script: {}".format(__version__))
print("[Version]class: {}".format(class_version))
print("[Version]function: {}".format(def_function.__version__))
print("[Paraments]counts_cutoff: {}".format(counts_cutoff))
print("[Paraments]annotation_col: {}".format(annotation_col))
print("[Paraments]quantification_col: {}".format(quantification_col))
print("[Paraments]input_sample_info: {}".format(input_sample_info))
print("[Paraments]input_gene_annotation_file: {}".format(input_gene_annotation_file))
print("[Paraments]sample_name_list: {}".format(sample_name_list))
print("[Paraments]output_df_filename: {}".format(output_df_filename))
print("[Paraments]output_pickle_filename: {}".format(output_pickle_filename))
print('\n')

In [18]:
# 准备input文件的绝对路径
file_path = def_function.get_file_path(file_path=file_path,
                                       sample_name_list=sample_name_list,
                                       tab_level=0)

In [19]:
# 准备存储gene信息
total = Total(sample_list=[], gene_dict={})

In [20]:
# 读取每个样本的数据
for sample_name in sample_name_list:
    quantification_file_location = file_path[sample_name]["quantification"]
    annotation_file_location = file_path[sample_name]["annotation"]

    total = def_function.load_sample(total,
                                     sample_name=sample_name,
                                     annotation_location=annotation_file_location,
                                     quantification_location=quantification_file_location,
                                     counts_cutoff=counts_cutoff,
                                     annotation_col=annotation_col,
                                     quantification_col=quantification_col,
                                     tab_level=0)



[Function]load_sample start.
	[Time]2023-08-07 14:58:09
	[Paraments]sample_name: GSM6783527
	[Paraments]annotation_location: F:/OneDrive/Master/Project/trans/data/raw_data/GSM6783527/GSM6783527_ENCFF384PHN_transcriptome_annotations_GRCh38.gtf
	[Paraments]quantification_location: F:/OneDrive/Master/Project/trans/data/raw_data/GSM6783527/GSM6783527_ENCFF816TJN_transcript_quantifications_GRCh38.tsv
	[Paraments]counts_cutoff: 2
	[Paraments]annotation_col: ['gene_id', 'transcript_id', 'gene_name']
	[Paraments]quantification_col: ['annot_gene_id', 'annot_transcript_id', 'annot_transcript_name']
	[Paraments]tab_level: 0
[2023-08-07 14:59:17]load_sample finished.


[Function]load_sample start.
	[Time]2023-08-07 14:59:17
	[Paraments]sample_name: GSM6782551
	[Paraments]annotation_location: F:/OneDrive/Master/Project/trans/data/raw_data/GSM6782551/GSM6782551_ENCFF856FNN_transcriptome_annotations_GRCh38.gtf
	[Paraments]quantification_location: F:/OneDrive/Master/Project/trans/data/raw_data/GSM67

In [21]:
# 读取基因组注释文件以补充gene/transcript的biotype信息
biotype_dict = def_function.get_biotype_info(annotation_file_location=input_gene_annotation_file,
                                             tab_level=0)
def_function.add_biotype_info(total, biotype_dict,
                              tab_level=0)



[Function]get_biotype_info start.
	[Time]2023-08-07 15:00:38
	[Paraments]annotation_file_location: F:/OneDrive/Master/Project/trans/data/raw_data/annotation/Homo_sapiens.GRCh38.109.gtf.gz
	[Paraments]tab_level: 0
[2023-08-07 15:00:45]get_biotype_info finished.
	[result]the counts of gene updated biotype: 15894
	[result]the counts of transcript updated biotype: 23365


True

In [22]:
# 计算每个transcript在每个样本中的相对表达值
total.computeRelativeExpression()

In [23]:
# 保存数据文件
df = def_function.save_df(total, output_df_filename,
                          tab_level=0)
with open(output_pickle_filename, 'wb') as file:
    pickle.dump(total, file)



[Function]save_df start.
	[Time]2023-08-07 15:00:46
	[Paraments]tab_level: 0
[2023-08-07 15:00:50]save_df finished.


---

调试

调试

---